# HMM classification - part 2

This doc lives in:

In [1]:
%%bash
pwd

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/docs


In the previous episode... workflow completed until the generation of fasta file with choosen 25 kunitz-containing sequences (after clustering and selection of representative sequences to avoid redundancies)

## Model generation in dumb way (sequence only)

Will be worth to demonstrate that taking into account the structural alignment allows to increase the alignment correctess, and consequently the classification performance of the resulting model

### Step1: run muscle for MSAing the 25 kunitz seqs:

In [2]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa
head chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
>1BUN_B
--------------------------RKRHPDCDK-PPDTKICQTVVRAFYYKPSAKRCV
QFRYGGCNGNGNHFKSDHLCRCECLEYR------------
>4NTW_B
--------------------------EIRPAFCYEDPPFFQKCGAFVDSYYFNRSRITCV
HFFYGQCDVNQNHFTTMSECNRVCHG--------------
>1DTX_A
--------------------------EPRRKLCIL-HRNPGRCYDKIPAFYYNQKKKQCE
RFDWSGCGGNSNRFKTIEECRRTCIG--------------
>5YV7_A


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	extract_representative_sequences
	1	filtered_intersection2fasta
	1	intersect_with_all_data
	1	intersection2fasta
	1	muscle_align
	1	parse_pdbefold_summary
	1	parse_rcsbpdb_table
	1	run_blastclust
	8

rule parse_pdbefold_summary:
    input: pdbefold_5pti_A_summary
    output: pdbefold_5pti_A_summary.txt.gz
    jobid: 6

cat pdbefold_5pti_A_summary | sed '1,5d' | tr -s " " "\t" | cut -f3- | awk '{print toupper($17)"	"$1"	"$2"	"$3"	"$4}' | sed 's/:/_/' | gzip > pdbefold_5pti_A_summary.txt.gz
Finished job 6.
1 of 8 steps (12%) done

rule parse_rcsbpdb_table:
    input: rcsbpdb_wt_PF00014_len50_90.csv
    output: rcsbpdb_wt_PF00014_len50_90.txt.gz
    jobid: 7

cat rcsbpdb_wt_PF00014_len50_90.csv | sed 's/,/	/g' | sed 's/"//g' | sed 's/ /_/g' | sed '1d' | sed 's/	/_/' | gzip > rcsbpdb_wt_PF00014_len50_90.txt.gz
Finished job 7.
2 of 8 steps (25%) done

rule intersect_with_all_data:
    input: rcsbpd

### Step2: run hmmbuild to build the model

In [3]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm
head chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa
# output HMM file:                  chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_a

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	build_hmm
	1

rule build_hmm:
    input: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm
    jobid: 0
    wildcards: msa=chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95, source=muscle

hmmbuild --informat afa chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa
Finished job 0.
1 of 1 steps (100%) done


\[classification and assessment of performance will be done later\]

## Multiple Structural Alignment
From [pdbefold](http://www.ebi.ac.uk/msd-srv/ssm/)

You can provide list of PDB entries: <PDBID\>:<CHAINID\>

Note: procedure may take some time. prof is showing this with 5-6 structures only

* Check manually the structures for each cluster to be problem proof before starting the alignment submission
* Submit the list here: http://www.ebi.ac.uk/msd-srv/ssm/cgi-bin/ssmserver
* Download FASTA alignment
* Parse to have everything uppercase

## hmmbuild on structure-derived sequence alignment

* run hmmbuild from the aligned fasta
* go to skylingn to get a nice visualization of the hmm profile: comment the conservation of typical residues etc; maybe compare this sequence logo with the one obtained from the non-structure-derived sequence alignment

First, get the list of pdb (identifier:chain)s

In [4]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_idlist

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	create_chosen_ids_list
	1

rule create_chosen_ids_list:
    input: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_idlist
    jobid: 0
    wildcards: sequences=chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95

cat chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.txt | cut -f1 | sed 's/_/:/' > chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_idlist
Finished job 0.
1 of 1 steps (100%) done


Then use it to feed pdbefold

**my session ID is 556-42-1FK**

## Choose the negative testing set

* Go to UniProt --> advanced search
* chose same length, non-kunitz, reviewed: yes

**length:[40 TO 500] NOT database:(type:pfam pf00014) AND reviewed:yes**

Downloaded in ../handmade/uniprot_kunitz_negative.gz

I renamed the file and deleted the header (first line: "Entry   Cross-reference (PDB)")

Sets generation for testing: prof sort(ed) -R | head -100

## Choose the positive testing set

* Go to UniProt --> advanced search
* search for 40-500L, kunitz, reviewed
* Exclude sequences that are too similar to the training set: possible approaches
    
**database:(type:pfam pf00014) length:[40 TO 500] AND reviewed:yes**

Downloaded in **../handmade/uniprot_kunitz_positives.gz**

I renamed the file and deleted the header (first line: "Entry   Cross-reference (PDB)")

In [5]:
%%bash
cd ../handmade/
ls -lh

total 150M
-rw-r--r-- 1 alessandro alessandro 3,3K apr  4 15:58 pdbefold_25_selected_kunitz.afa
-rw-r--r-- 1 alessandro alessandro  78K apr  3 12:58 pdbefold_5pti_A_summary
-rw-r--r-- 1 alessandro alessandro 147M apr  3 11:53 pdb_seqres.txt
-rw-r--r-- 1 alessandro alessandro  18K apr  3 11:28 rcsbpdb_wt_PF00014_len50_90_addinfo.csv
-rw-r--r-- 1 alessandro alessandro  16K apr  3 12:20 rcsbpdb_wt_PF00014_len50_90.csv
-rw-r--r-- 1 alessandro alessandro 2,0M apr  4 18:03 uniprot_kunitz_negative.gz
-rw-r--r-- 1 alessandro alessandro 1,6K apr  4 18:38 uniprot_kunitz_positive.gz


In the meantime, MSA of structure finished. I downloaded the fasta alignment (afa) here:

In [6]:
%%bash
head ../handmade/pdbefold_25_selected_kunitz.afa

>PDB:5pti:A STRUCTURE OF BOVINE PANCREATIC TRYPSIN INHIBITOR. 
--------rpdfcleppytgpckARIIRYFYNAKAGLCQTFVYGgCRA-KRNNFKSAEDCMRTCgga-

>PDB:1aap:A X-RAY CRYSTAL STRUCTURE OF THE PROTEASE INHIBITOR 
--------vrevcseqaetgpcrAMISRWYFDVTEGKCAPFFYGgCGG-NRNNFDTEEYCMAVCg---

>PDB:3byb:A CRYSTAL STRUCTURE OF TEXTILININ-1, A KUNITZ-TYPE S
------kdrpdfcelpadtgpcrVRFPSFYYNPDEKKCLEFIYGgCEG-NANNFITKEECESTCa---

>PDB:3wny:A A SIMPLIFIED BPTI VARIANT WITH POLY LYS AMINO ACID


Symbolic links to the new handmade files are now present in ../dataset/

In [7]:
%%bash
cd ../dataset/
ls -lh

total 112M
-rw-r--r-- 1 alessandro alessandro  809 apr  4 20:10 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.fasta.gz
-rw-r--r-- 1 alessandro alessandro  175 apr  4 20:10 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_idlist
-rw-r--r-- 1 alessandro alessandro 2,7K apr  4 20:10 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.afa
-rw-r--r-- 1 alessandro alessandro  29K apr  4 20:10 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.muscle.hmm
lrwxrwxrwx 1 alessandro alessandro   43 apr  4 16:11 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.afa -> ../handmade/pdbefold_25_selected_kunitz.afa
-rw-r--r-- 1 alessandro alessandro 2,9K apr  4 20:10 chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_p

## Building HMM with structure-derived sequence alignment

In [8]:
%%bash
cd ../dataset/
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm
head chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.afa
# output HMM file:                  chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold    25    68    59     2.43  0.946 

# CPU time

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	build_hmm
	1

rule build_hmm:
    input: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.afa
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm
    jobid: 0
    wildcards: msa=chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95, source=pdbefold

hmmbuild --informat afa chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.afa
Finished job 0.
1 of 1 steps (100%) done


## Filtering the positive set for similarity with the training set
* best way: use blastclust to remove some extremely similar clusters
* fast, ugly alternative: remove only the 25 training sequences

Here I do the best way:

In [9]:
%%bash
cd ../dataset/
pwd
snakemake -p filtered_uniprot_kunitz_positive.txt
cat filtered_uniprot_kunitz_positive.txt | head
wc -l filtered_uniprot_kunitz_positive.txt

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
P00981
P00982
P00978
Q64240
P04366
O95925
P68425
Q9WU03
P29216
Q07456
310 filtered_uniprot_kunitz_positive.txt


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	filter_out_training_clusters
	1

rule filter_out_training_clusters:
    input: intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz, uniprot_kunitz_positive.gz
    output: filtered_uniprot_kunitz_positive.txt
    jobid: 0

zcat uniprot_kunitz_positive.gz | grep -v -f <(zcat intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95_blastclust_out.gz | sed '1d' | sed 's/ $//g' | sed 's/ /\n/g' | cut -b 1-4 | sort | uniq) | cut -f1 > filtered_uniprot_kunitz_positive.txt
Finished job 0.
1 of 1 steps (100%) done


## Retrieve multifasta testing sets
### positive:

In [10]:
%%bash
cd ../dataset/
pwd
snakemake -p filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz
zcat filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz | head

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
>sp|P84875|PCPI_SABMA Carboxypeptidase inhibitor SmCI OS=Sabellastarte magnifica OX=389514 PE=1 SV=1
ISVCDLPADRGQCTAYIPQWFFNKTTEDCEKFVYGGCQGNANRFETKDDCIANCGCNLPS
KVGPCRVSARMWFHNPETEKCEVFIYGGCHGNANRFATETECQEVCDRYQKPGFCYQPSE
TGPCKGSFPRYYYDYEDGECKEFIYGGCEGNANNFETKESCENAC
>sp|Q9D263|SPIT4_MOUSE Kunitz-type protease inhibitor 4 OS=Mus musculus OX=10090 GN=Spint4 PE=2 SV=1
MKPTKLGFLLGLSLLCSLSPPVLSGVERLANYLCKDYNDPCLLDVEPGSCYEVHFRFFYN
QTAKQCQIFLFTGCNGNLNNFKLKIDCDVTCHETYKNPPLPSGDKRKRSLRALATKNLAT
DWRLTTLQPGRLRQTERKETLLQRARRESRRQPLARHHT
>sp|Q9WU03|SPIT2_MOUSE Kunitz-type protease inhibitor 2 OS=Mus musculus OX=10090 GN=Spint2 PE=1 SV=1
MAQLCELRRGRALLALVASLLLSGAQVASRELDVHESCGVSKVVGKCRASIPRWWYNITD


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	filterfasta
	1

rule filterfasta:
    input: swissprot.fasta.gz, filtered_uniprot_kunitz_positive.txt
    output: filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz
    jobid: 0
    wildcards: filter=filtered_uniprot_kunitz_positive, db=swissprot

python ../src/fastafilter.py swissprot.fasta.gz filtered_uniprot_kunitz_positive.txt | gzip > filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz
Finished job 0.
1 of 1 steps (100%) done


### Negative:

In [11]:
%%bash
cd ../dataset/
pwd
snakemake -p filtered_uniprot_kunitz_negative.txt
head filtered_uniprot_kunitz_negative.txt

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
Q00604
Q62433
P13738
P01179
P25427
P55769
Q9D0T1
Q4PSE2
Q92886
P80188


Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	get_negative_set_ids
	1

rule get_negative_set_ids:
    input: uniprot_kunitz_negative.gz
    output: filtered_uniprot_kunitz_negative.txt
    jobid: 0

zcat uniprot_kunitz_negative.gz | cut -f1 > filtered_uniprot_kunitz_negative.txt
Finished job 0.
1 of 1 steps (100%) done


## Run hmmsearch
### Positive set:

In [12]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_filtered_uniprot_kunitz_positive_filter_on_swissprot.search_out.gz
zcat chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_filtered_uniprot_kunitz_positive_filter_on_swissprot.search_out.gz | head -20

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm
# target sequence database:        /dev/fd/63
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold  [M=59]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence           

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	hmm_db_search
	1

rule hmm_db_search:
    input: filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_filtered_uniprot_kunitz_positive_filter_on_swissprot.search_out.gz
    jobid: 0
    wildcards: model=chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold, db=filtered_uniprot_kunitz_positive_filter_on_swissprot

hmmsearch chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm <(zcat filtered_uniprot_kunitz_positive_filter_on_swissprot.fasta.gz) | gzip > chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_filtered_uniprot_kunitz_positive_filter_on_swissprot.search_out.gz
Finished job 0.
1 

### Negative set:

In [13]:
%%bash
cd ../dataset/
pwd
snakemake -p chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_swissprot.search_out.gz
zcat chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_swissprot.search_out.gz | head -20

/home/alessandro/Unibo/python-programming-alessandro-lussana/LB1/prj_hmm_classification/dataset
# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm
# target sequence database:        /dev/fd/63
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold  [M=59]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence           

Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	hmm_db_search
	1

rule hmm_db_search:
    input: swissprot.fasta.gz
    output: chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_swissprot.search_out.gz
    jobid: 0
    wildcards: model=chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold, db=swissprot

hmmsearch chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold.hmm <(zcat swissprot.fasta.gz) | gzip > chosen_blastclust_seq_for_intersec_rcsbpdb_wt_PF00014_len50_90_and_pdbefold_5pti_A_summary_S99_L0.95.pdbefold_vs_swissprot.search_out.gz
Finished job 0.
1 of 1 steps (100%) done


## Parse the hmmsearch results
Keep the identifier and one of the e-value (test different values and their meaning in order to maximize the classificator performance)

### Notes of e-value operative interpretation
e-val is a function of the dataset size; if you hmmsearch a dataset that is made but only 1 sequence: you obtain an e-value that is x times higher than what you get in a database of x sequences --> due to Bonferroni Correction

If you divide for the size of the datasets, you should be able to compare two searches

## Classification of negatives
You need to pass option not to neglet the non-hits: --max

--max    : Turn all heuristic filters off (less speed, more power) 

or

to scale the threshold, remove the e-value cutoff:

--domE <x> : report domains <= this E-value threshold in output  [10.0]  (x>0)
    
(domE works for the domain e-value, there is also an option for the full sequence)

**If you don't want the alignment: hmmsearch --noali**

## Assessment of the performance
* Correction of e-value
* parse to have
    * id
    * score
    * class (positive: 1; negative: 0)
* feed the python script for computing the confusion_matrix with the info in point 2

**For the report**: 
* TODO put some equations showing some assessment scores from the confusion matrix
* TODO use also tcoffee for structure-based MSA and compare the HMMs
* TODO automatize the threshold optimization (maybe with scikit-learn)
* TODO plot ROC and compute the area under ROC